In [5]:
from dotenv import load_dotenv
import os

In [ ]:
# Load the .env file
load_dotenv()

In [7]:
# Access the variable
tmdb_api_key = os.getenv('TMDB_API_KEY')

# print(tmdb_api_key)  # This will print: tmd_api_key

Rated Movies
GET
https://api.themoviedb.org/3/account/{account_id}/rated/movies

In [8]:
import requests

url = "https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page=1&sort_by=popularity.desc&vote_average.gte=8&vote_count.gte=1000"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_key}"
}

movie_response = requests.get(url, headers=headers).json()

In [9]:
url = "https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page=2&sort_by=popularity.desc&vote_average.gte=8&vote_count.gte=1000"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_key}"
}

movie_response2 = requests.get(url, headers=headers).json()
movie_response['results'] += movie_response2['results']

In [10]:
url = "https://api.themoviedb.org/3/discover/movie?include_adult=false&include_video=false&language=en-US&page=3&sort_by=popularity.desc&vote_average.gte=8&vote_count.gte=1000"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_key}"
}

movie_response3 = requests.get(url, headers=headers).json()
movie_response['results'] += movie_response3['results']

In [11]:
secure_base_url = "https://image.tmdb.org/t/p/"
poster_sizes = ["w92", "w154", "w185", "w342", "w500", "w780", "original"]
use_poster_size = poster_sizes[2]
backdrop_sizes = ["w300", "w780", "w1280", "original"]
use_backdrop_size = backdrop_sizes[1]
poster_base_url = secure_base_url + use_poster_size
backdrop_base_url = secure_base_url + use_backdrop_size

In [12]:
url = "https://api.themoviedb.org/3/configuration/languages"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_key}"
}

language_response = requests.get(url, headers=headers).json()

In [13]:
# print(language_response)
languages = dict()
for language in language_response:
    languages[language['iso_639_1']] = language['english_name']
# print(languages)

In [14]:
movie_lst = []

for movie in movie_response['results']:
    # print(movie['title'], movie['vote_average'], movie['vote_count'])
    new_row = {'title': movie['title'], 'plot_summary': movie['overview'],'release_date': movie['release_date'], 'poster_url': poster_base_url + movie['poster_path'], 'backdrop_url': backdrop_base_url + movie['backdrop_path'], 'language': languages[movie['original_language']], 'tmdb_id': movie['id'], 'genres': movie['genre_ids']}

    url = f"https://api.themoviedb.org/3/movie/{movie['id']}?language=en-US"

    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {tmdb_api_key}"
    }

    specific_movie_response = requests.get(url, headers=headers).json()
    new_row['duration_in_mins'] = specific_movie_response['runtime']

    movie_lst.append(new_row)
# print(df)


In [16]:
import pandas as pd

In [17]:
df = pd.DataFrame(movie_lst, columns=['title', 'plot_summary', 'release_date', 'poster_url', 'backdrop_url', 'language', 'duration_in_mins'])
# Write the DataFrame to a CSV file
df.to_csv('movie.csv', index=False)

Get MoviePerson details

Take top 8 casts from each movie

In [25]:
cast_limit = 8
people_ids = set()
movie_to_cast = dict()
movie_to_director = dict()

for movie in movie_lst:
    url = f"https://api.themoviedb.org/3/movie/{movie['tmdb_id']}/credits"

    headers = {
        "accept": "application/json",   
        "Authorization": f"Bearer {tmdb_api_key}"
    }
    
    casts_response = requests.get(url, headers=headers).json()

    movie_to_director[movie['title']] = [{'name': crew['name'], 'tmdb_id': crew['id']} for crew in casts_response['crew'] if crew['job'] == 'Director']
    for director in movie_to_director[movie['title']]:
        people_ids.add(director['tmdb_id'])

    if movie['title'] not in movie_to_cast:
        movie_to_cast[movie['title']] = []

    for cast in casts_response['cast']:
        if cast['order'] >= cast_limit:
            break
        people_ids.add(cast['id'])
        movie_to_cast[movie['title']].append({'name': cast['name'], 'character': cast['character'], 'order': cast['order'], 'tmdb_id': cast['id']})

print(len(people_ids)) 

466


In [27]:
people_lst = []
for people_id in people_ids:
    url = f"https://api.themoviedb.org/3/person/{people_id}"

    headers = {
        "accept": "application/json",   
        "Authorization": f"Bearer {tmdb_api_key}"
    }

    people_response = requests.get(url, headers=headers).json()

    new_row = {
        'name': people_response.get('name', ''),
        'date_of_birth': people_response.get('birthday', ''),
        'biography': people_response.get('biography', ''),
        'image_url': poster_base_url + (people_response.get('profile_path') or ''),
        'date_of_death': people_response.get('deathday', ''),
        'tmdb_id': people_response.get('id', ''),
        'place_of_birth': people_response.get('place_of_birth', '')
    }

    people_lst.append(new_row)

print(len(people_lst))

466


In [29]:
df = pd.DataFrame(people_lst, columns=['date_of_birth', 'biography', 'image_url', 'date_of_death', 'name', 'place_of_birth'])
df.to_csv('movie_person.csv', index=False)

Get all genres and create the genre table

In [18]:
url = "https://api.themoviedb.org/3/genre/movie/list?language=en"

headers = {
    "accept": "application/json",
    "Authorization": f"Bearer {tmdb_api_key}"
}

genres_response = requests.get(url, headers=headers).json()

In [19]:
genre_dict = dict()
genre_lst = []
for genre in genres_response['genres']:
    genre_dict[genre['id']] = genre['name']
    genre_lst.append({'name': genre['name']})

print(len(genre_lst))

19


In [20]:
df = pd.DataFrame(genre_lst, columns=['name'])
df.to_csv('genre.csv', index=False)

Load supabase db

In [21]:
from supabase import create_client, Client

url: str = os.getenv("SUPABASE_URL")
key: str = os.getenv("SUPABASE_ANON_KEY")
supabase: Client = create_client(url, key)

Create movie_mapping from supabase

In [30]:
supa_movie = dict()

response = supabase.table('movie').select('id, title').execute()

for movie in response.data:
    supa_movie[movie['title']] = movie['id']

# print(supa_movie)

2024-02-06 08:43:03,365:INFO - HTTP Request: GET https://yvezrqitxosjfitjgzmz.supabase.co/rest/v1/movie?select=id%2C%20title "HTTP/1.1 200 OK"


Create movie_person mapping from supabase

In [31]:
supa_movie_person = dict()

response = supabase.table('movie_person').select('id, name').execute()

for person in response.data:
    supa_movie_person[person['name']] = person['id']

# print(supa_movie_person)

2024-02-06 08:43:08,229:INFO - HTTP Request: GET https://yvezrqitxosjfitjgzmz.supabase.co/rest/v1/movie_person?select=id%2C%20name "HTTP/1.1 200 OK"


Create genre mapping from supabase

In [32]:
supa_genre = dict()

response = supabase.table('genre').select('id, name').execute()

for genre in response.data:
    supa_genre[genre['name']] = genre['id']

# print(supa_genre)

2024-02-06 08:43:11,699:INFO - HTTP Request: GET https://yvezrqitxosjfitjgzmz.supabase.co/rest/v1/genre?select=id%2C%20name "HTTP/1.1 200 OK"


Create the relationship tables

Create movie_has_genre, movie_has_cast, movie_has_director

In [33]:
movie_has_genre_lst = []
movie_has_cast_lst = []
movie_has_director_lst = []

for movie in movie_lst:
    movie_tmdb_id = supa_movie[movie['title']]
    for genre_id in movie['genres']:
        movie_has_genre_lst.append({'movie_id': movie_tmdb_id, 'genre_id': supa_genre[genre_dict[genre_id]]})
    
    for cast in movie_to_cast[movie['title']]:
        movie_has_cast_lst.append({'movie_id': movie_tmdb_id, 'movie_person_id': supa_movie_person[cast['name']], 'role_name': cast['character'], 'order': cast['order']})
    
    for director in movie_to_director[movie['title']]:
        movie_has_director_lst.append({'movie_id': movie_tmdb_id, 'movie_person_id': supa_movie_person[director['name']]})

In [34]:
df = pd.DataFrame(movie_has_genre_lst, columns=['movie_id', 'genre_id'])
df.to_csv('movie_has_genre.csv', index=False)

In [35]:
df = pd.DataFrame(movie_has_cast_lst, columns=['movie_id', 'movie_person_id', 'role_name', 'order'])
df.to_csv('movie_has_cast.csv', index=False)

In [36]:
df = pd.DataFrame(movie_has_director_lst, columns=['movie_id', 'movie_person_id'])
df.to_csv('movie_has_director.csv', index=False)

Create movie_has_images

In [38]:
movie_has_images_lst = []
img_lim = 7

for movie in movie_lst:
    movie_tmdb_id = movie['tmdb_id']
    movie_supa_id = supa_movie[movie['title']]

    url = f"https://api.themoviedb.org/3/movie/{movie_tmdb_id}/images"

    headers = {
        "accept": "application/json",
        "Authorization": f"Bearer {tmdb_api_key}"
    }

    images_response = requests.get(url, headers=headers).json()

    cnt = 0
    for image in images_response['posters']:
        movie_has_images_lst.append({'movie_id': movie_supa_id, 'image_url': poster_base_url + image['file_path'], 'image_type': 'poster', 'aspect_ratio': image['aspect_ratio']})
        cnt += 1
        if cnt >= img_lim:
            break
    
    cnt = 0
    for image in images_response['backdrops']:
        movie_has_images_lst.append({'movie_id': movie_supa_id, 'image_url': backdrop_base_url + image['file_path'], 'image_type': 'backdrop', 'aspect_ratio': image['aspect_ratio']})
        cnt += 1
        if cnt >= img_lim:
            break
    

In [39]:
df = pd.DataFrame(movie_has_images_lst, columns=['movie_id', 'image_url', 'image_type', 'aspect_ratio'])
df.to_csv('movie_has_images.csv', index=False)